# Stock Portfolio Optimal Adjust
Un inversor propietario de una cartera de valores busca maximizar los ingresos por dividendos a un año vista.

Dada una predicción del precio de los valores de su cartera, el inversor pretende ajustar su cartera buscando la máxima rentabiliadd posible al capital invertido.

En concreto se enfrenta a la decisión sobre el número de acciones que debería vender o comprar de cada valor para maximizar sus beneficios y al mismo tiempo incrementar el valor de su cartera

# Data

## Conjuntos
* $\Omega$: Conjunto de Valores bursalites
* $M = |\Omega|$ : número de valores bursatiles

## Parámetros
* $b_i$: número actual de participaciones del valor bursatil $i$
* $v_i$: el precio actual valor $i$ por acción
* $w_i$: nuevo precio del valor $i$ al final del año
* $d_i$: dividendo a pagar al final del año del valor bursatil $i$

## Constantes
* $r$: Peso máximo que puede tener cada participación de cada valor en la cartera tras el ajuste
* $s$: Incremento mínimo de valor de la cartera tras el ajuste


## Variables

* $x_i$ Cambio en el número de acciones de cada valor bursatil $i \in \Omega$


# Modelo
## Función Objetivo
Maximizar los dividendos

$$
z = \sum_{i\in \Omega} d_i (b_i + x_i)
$$

## Restricciones

### Restricción 1
Valor de las participaciones debe ser no negativo
$$
x_i \ge -b_i, \hspace{0.3cm} \forall i \in \Omega
$$

### Restricción 2
La cartera debe evitar depender en exceso de un valor cualquiera tras el ajuste

$$
v_j(b_j+x_j)\leq r  \left( \sum_{i \in \Omega} v_i(b_i + x_i) \right) , \hspace{0.3cm} \forall j \in \Omega
$$

### Restricción 3
El capital total de la cartera no debe cambiar tras el ajuste (no se invierte dinero adicional, es decir, las acciones que se compran, se pagan con la venta de otras)


$$
\sum_{i \in \Omega} v_i  x_i = 0
$$



### Restricción 4
El capital total en el futuro debe ser al menos cierto porcentaje mayor que el capital invertido en el momento presente

$$
\sum_{i \in \Omega} w_i(bi+x_i) \ge (1+s) \sum_{i \in \Omega} v_i b_i
$$




# Modelo Pyomo

In [14]:
import pyomo.environ as pyo
from pyomo.opt import SolverFactory, SolverStatus, TerminationCondition
import numpy as np
np.random.seed(0)

## Model Data

In [405]:
r = 0.25
s = 0.01
stocks = ['AAPL', 'AMZN', 'TSLA']
actual_stock  = {'AAPL':75, 'AMZN':100, 'TSLA':35}
actual_prices = {'AAPL':20, 'AMZN':20,  'TSLA':100}
future_prices = {'AAPL':18, 'AMZN':23,  'TSLA':195}
dividendos    = {'AAPL':0,  'AMZN':5,   'TSLA':5}

#future_prices = {i:round(actual_prices[i]* np.random.uniform(0.98, 1.05)) for i in stocks}
#future_prices
actual_investment = sum(actual_stock[i]*actual_prices[i] for i in stocks)
expected_profit = sum(actual_stock[i]*dividendos[i] for i in stocks)
print('Actual investment: {} euros'.format(actual_investment))
print('Expected profit: {} euros'.format(expected_profit))


Actual investment: 7000 euros
Expected profit: 675 euros


In [408]:
# model
model = pyo.ConcreteModel()
# Indexes
#model.i = pyo.RangeSet(1, N)
model.stocks  = pyo.Set(initialize=stocks)
# Variables
model.x = pyo.Var(model.stocks, domain=pyo.Reals, initialize=0)  # PositiveReals PositiveIntegers

# Parameters
# actual price
model.v = pyo.Param(model.stocks, initialize=actual_prices, within=pyo.Any)
# actual no. stocks
model.b = pyo.Param(model.stocks, initialize=actual_stock, within=pyo.Any)
# future price
model.w = pyo.Param(model.stocks, initialize=future_prices, within=pyo.Any)
# expected dividends
model.d = pyo.Param(model.stocks, initialize=dividendos, within=pyo.Any)


# Constraint 1
def constraint_1(model, i):
    return model.x[i] >= -model.b[i]
model.constraint_1 = pyo.Constraint(model.stocks, rule=constraint_1)

# Constraint 2
def constraint_2(model, j):
    return model.v[j]*(model.b[j]+model.x[j]) >= r* sum(model.v[i]*(model.b[i]+model.x[i]) for i in model.stocks )
model.constraint_2 = pyo.Constraint(model.stocks, rule=constraint_2)

# Constraint 3
def constraint_3(model):
    return sum(model.v[i]*model.x[i] for i in model.stocks) ==0 
model.constraint_3 = pyo.Constraint(model.stocks, rule=constraint_3)

# Expresion 1 (future value of the investment)
def expresion_1(model):
    return sum(model.w[i] * model.b[i] for i in model.stocks)
model.future_invest = pyo.Expression(rule=expresion_1)

# Expresion 3 (future value of the investment)
def expresion_3(model):
    return sum(model.w[i] * (model.b[i]+model.x[i]) for i in model.stocks)
model.future_invest_with_ad = pyo.Expression(rule=expresion_3)

# Expresion 2 (present value of the investment)
def expresion_2(model):
    return sum(model.v[i] * model.b[i] for i in model.stocks)
model.actual_invest = pyo.Expression(rule=expresion_2)

# Expresion 2 (present value of the investment after adjustment)
def expresion_4(model):
    return sum(model.v[i] * (model.b[i]+model.x[i]) for i in model.stocks)
model.actual_invest_with_ad = pyo.Expression(rule=expresion_4)

# Constraint 4
def constraint_4(model):
    return model.future_invest_with_ad  >= (1+s)* model.actual_invest
model.constraint_4 = pyo.Constraint(rule=constraint_4)

# Objective
def objective_rule(model):
    profit = sum(model.d[i]*(model.x[i]+model.b[i]) for i in model.stocks)
    return profit
model.OF = pyo.Objective(rule=objective_rule, sense=pyo.maximize, doc='maximize profit')




In [410]:

path_cplex = '/Applications/CPLEX_Studio_Community201/cplex/bin/x86-64_osx/cplex'
path_gurobi=  '/usr/local/bin/gurobi.sh'

#opt = SolverFactory("gurobi", solver_io="python")  # glpk
#opt = SolverFactory("cplex", solver_io="python", executable=path_cplex)  # glpk
opt = SolverFactory("glpk")  # glpk


results = opt.solve(model)  # tee=True for details

# Check the solution
if (results.solver.status == SolverStatus.ok) and (results.solver.termination_condition == TerminationCondition.optimal):
    print('--- Termination Condition ---')
    print('    {}'.format(results.solver.termination_condition))
    print('\n')
    print('--- Results ---')
    #print('Feasible solution')
    #model.OF.pprint()
    optimal_profit = pyo.value(model.OF)
    print('    Optimal profit: {}'.format(optimal_profit))
    print('    Expected profit: {}'.format(expected_profit))
    print('    Profit GAP: {}'.format(optimal_profit - expected_profit ))
    print('    Actual Investment without adjustment: {}'.format(pyo.value(model.actual_invest)))
    print('    Actual Investment with adjustment: {}'.format(pyo.value(model.actual_invest_with_ad)))
    print('    Future Investment without adjustment: {}'.format(pyo.value(model.future_invest)))
    print('    Future Investment with adjustment: {}'.format(pyo.value(model.future_invest_with_ad)))
    sp = 100*round((pyo.value(model.future_invest_with_ad) - pyo.value(model.actual_invest)) / pyo.value(model.actual_invest),4)
    print('    Investment GAP: {} ({} %)'.format(pyo.value(model.future_invest_with_ad) - pyo.value(model.actual_invest), sp))
    
    for stock in model.stocks:
        print('    {}: {}'.format(stock, pyo.value(model.x[stock])))


    print('--- Problem Results ---')
    print(results.Problem())
    #print('--- Solver Results ---')
    print('Execution time: {}'.format(results.Solver().Time))
elif results.solver.termination_condition == TerminationCondition.infeasible:
    print('termination condition:', results.solver.termination_condition)
    print('(!) Infeasible solution')
else:
    #
    # Something else is wrong
    print('--- Termination Condition ---')
    print('  (!) {}'.format(results.solver.termination_condition))
    print('\n')
    model.pprint()
    #print('Status:', results.solver.status)

--- Termination Condition ---
    optimal


--- Results ---
    Optimal profit: 962.5
    Expected profit: 675
    Profit GAP: 287.5
    Actual Investment without adjustment: 7000.0
    Actual Investment with adjustment: 7000.0
    Future Investment without adjustment: 10475.0
    Future Investment with adjustment: 9012.5
    Investment GAP: 2012.5 (28.749999999999996 %)
    AAPL: 12.5
    AMZN: 75.0
    TSLA: -17.5
--- Problem Results ---

Name: unknown
Lower bound: 962.5
Upper bound: 962.5
Number of objectives: 1
Number of constraints: 11
Number of variables: 4
Number of nonzeros: 25
Sense: maximize

Execution time: 0.027069807052612305


# Referencias

[1]. [Formulación y resolucuión de Modelos de Programación Matemática en Ingeniería y Ciencia](http://www.dia.fi.upm.es/~jafernan/teaching/operational-research/LibroCompleto.pdf)